1. Import openAI dependences (load with conda install openai)

In [1]:

#Import openAI dependences (module)
import openai
import csv
import re

2. Use the API Key from openAI page

In [6]:
#Add our API Key copied from openAI page
openai.api_key = "" #Example -> dw-bQyr9128F49VJRNKAnST32lbkFJ2Zv4qEhWo4R0Xh6yrh15

3. Make a request

In [7]:
#Make the context of our prompt envoirement
context = 'Eres un experimentado tester que analiza requerimientos funcionales y genera casos de prueba. El sistema que se esta desarrollando es para trazabilidad de medicamentos y esta apuntado a clientes muy exigentes'

#prompt (text/user's question)
prompt = """
Como parte del desarrollo en Verifarma 2.0, se requiere completar la segunda parte del Serie Generator Rules con el objetivo de generar la serie completa de manera única. Esta tarea es continuación de la primera parte abordada en los jiras VF2-436: [SNG][FRONTEND] Servicio Generación de series - Configuracion de Supported characters y Fragment codes
IN TEST
  y VF2-439: [SNG][FRONTEND] Servicio Generación de Permutaciones - Implementación Generator Rules
IN TEST
 . Ahora se requiere la creación de un nuevo formulario para la identificación de productos (commodity identification), conocido en otros proyectos como medicamentos/productos.

Una vez determinado la arquitectura a utilizar y ver si es posible reutilizar el front-end del Exchanger/FarmaDigit o si es necesario crear una nueva interfaz, este enfoque debe ser aplicado al nuevo proceso de alta para la gestión de este ABM (Alta/Baja/Modificación) de productos.

Alcance del Requerimiento
El objetivo principal de este requerimiento es desarrollar un front-end que permita a los usuarios realizar las siguientes acciones relacionadas con el Commodity Identification (Identificación de Producto):

 ABM-Producto/commodity Identification (Alta/Modificar/Eliminar):

Implementar funcionalidades para dar de alta, modificar y eliminar registros de productos en el sistema.

Cluster Size:

Definir y gestionar el tamaño del clúster (grupo o conjunto) asociado con la identificación del producto, según los requisitos del negocio.

Botón "Generar Series" e Invocación del Método:

Incorporar un botón en la interfaz de usuario que permita a los usuarios generar series de identificación de productos y la correspondiente invocación del método necesario para este proceso.

Interacción con Serie Generator Rules parte 1:

Establecer la interacción adecuada entre los caracteres admitidos (Supported Characters) y el código de fragmento (Fragment Code) asociados con la identificación del producto, asegurando la coherencia y validez de los datos ingresados.


IMPORTANTE!!

Los datos que se en el esta pantalla debe ser visualizable a nivel de tenant, con la intención que al momento de dar de alta el producto 1 de la empresa A, dicho producto no pueda ser visualizado por la empresa B 

Fuera del Alcance
las concatenación de la series

Solución
Commodity Identification

El Commodity Identification se refiere al proceso de representar un producto mediante un valor único. Como parte de la solución, se requiere desarrollar un nuevo formulario para el ABM (Alta/Baja/Modificación) del Commodity Identification. Este formulario debe incluir los siguientes campos esenciales:

Nombre del Producto

Número de Identificación

Cluster Size

Además, la pantalla debe cumplir con las siguientes acciones correspondientes al ABM:

Agregar/Alta: Permite dar de alta un nuevo producto en el sistema.

Editar/Modificar: Permite modificar el nombre del producto y cluster. El número de identificación no será editable.

Eliminar: Esta acción desactivará el producto en lugar de eliminarlo físicamente del sistema.

Filtros de Búsqueda

Cluster Size: Este campo permite al usuario indicar manualmente la cantidad de series que se guardarán en un archivo dentro del sistema S3.

Botón de Solicitud de Series: Esta funcionalidad permite a los usuarios iniciar el proceso de generación y concatenación de series de acuerdo con las reglas específicas del producto.

Ejemplo de visualización:


Nombre 

Nro de Identificación

Cluster_size

Aspirina

123456

100

Ibuprofeno

963852

5.000


Notas:
El número de identificación no puede ser editado directamente en la pantalla.

La eliminación desactiva el producto en lugar de eliminarlo físicamente del sistema para mantener el historial y la integridad de los datos.

Esta solución proporcionará una interfaz clara y funcional para la gestión eficiente del Commodity Identification dentro del sistema, asegurando la coherencia y usabilidad de la aplicación.

Implementación
Interacción Con el ‘FC’ y ‘SC’ (Fragment Code y Supported Characters)
Esta funcionalidad permite a los usuarios definir reglas de serialización para un Producto existente mediante la configuración de Fragment Codes (FC) y Supported Characters (SC) dentro del módulo Serie Generator Rules en Jira VF2-439: [SNG][FRONTEND] Servicio Generación de Permutaciones - Implementación Generator Rules
IN TEST
 
Pasos y Detalles
Una vez que el usuario haya dado de alta un commodity identification en el sistema, podrá relacionarlo de la siguiente manera:

Acceso a la Implementación de Serie Generator Rules:

El usuario debe acceder a la pantalla de las reglas de generación de series desarrollada en el Jira: VF2-439.

Selección de 'FC' y 'SC':

Dentro del módulo Serie Generator Rules, el usuario puede seleccionar los Fragment Codes y Supported Characters  como parte a definir  de las reglas de serialización del Producto.

Vinculación con Commodity Identification (CI):

el usuario debe vincular los Fragment Codes y Supported Characters seleccionados con un Commodity Identification (CI) específico.

Especificación del Cluster Size (CS):

El usuario indica el tamaño del cluster (Cluster Size) requerido para la generación y agrupación de las series conforme a las reglas definidas.

Definición Completa de Reglas de Serialización:

Al completar la configuración de Fragment Codes, Supported Characters, Commodity Identification y Cluster Size, se establecen las reglas completas de serialización para el Producto.

Relación en el Back

Una vez el ‘CI’ se vincule para que llegue a ser parte del Serie Generator Rules, debe actualizarse el campo de la tabla Fragment_codes.Commodity_identification_id 

Click en Generar serie

Dispara la petición de serie que a posteriori se creara en una orden de serialización

Resultado
Mensaje de salida para kafka tipo Json que tiene como resultado la creación de una orden de serialización
 

Criterios de Aceptación
CA1:Alta del Commodity Identification

CA2: Modificación/eliminación del ‘CI’

C3: Vinculación del ‘CI’ al Serie Generator Rules
"""

4. Settings

In [ ]:
def analizar_casos_prueba(requerimiento):
    casos = re.findall(r'Objetivo: (.*?)\nPasos: (.*?)\nResultado esperado: (.*?)\n', requerimiento, re.DOTALL)
    return casos

def guardar_casos_csv(casos, nombre_archivo):
    with open(nombre_archivo, mode='w', newline='') as archivo:
        escritor_csv = csv.writer(archivo)
        escritor_csv.writerow(['Objetivo', 'Pasos', 'Resultado esperado'])
        for caso in casos:
            escritor_csv.writerow(caso)

5. Storage the response in any variable

In [ ]:
casos_prueba = analizar_casos_prueba(requerimiento)

guardar_casos_csv(casos_prueba, 'casos_de_prueba.csv')

print("Casos de prueba guardados en 'casos_de_prueba.csv'")
